In [1]:
# Essential Libraries
import os
import sys
import numpy as np
import pandas as pd
import datetime
import random

# ML
from sklearn import datasets
import tensorflow as tf
import keras

# Plots
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
sns.set_style("whitegrid");
sns.set_color_codes(palette='pastel');

# Stats
from scipy.stats import skew, norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

# Display options
pd.set_option('display.float_format', lambda x: '%.3f' % x);
pd.set_option('display.max_columns', None);
pd.options.display.max_seq_items = 8000;
pd.options.display.max_rows = 8000;

# Ignore Warnings
import warnings
warnings.filterwarnings(action="ignore")

In [2]:
# Define data and image directory
PROJECT_ROOT_DIR = "."
DATA_DIR = os.path.join(PROJECT_ROOT_DIR, "data")
IMAGE_DIR = os.path.join(PROJECT_ROOT_DIR, "img")

from IPython.display import Image
for d in [DATA_DIR, IMAGE_DIR]:
    if not os.path.exists(d):
        os.mkdir(d)

# Define a function to save images
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGE_DIR, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
from pprint import pprint

# This is a little recursive helper function converts all nested ndarrays to python list of lists so that pretty printer knows what to do.

def arrayToList(arr):
    if type(arr) == type(np.array):
        #If the passed type is an ndarray then convert it to a list and
        #recursively convert all nested types
        return arrayToList(arr.tolist())
    else:
        #if item isn't an ndarray leave it as is.
        return arr
    
# Suppress exponential notation, define an appropriate float formatter
# Specify stdout line width and let pretty print do the work
# np.set_printoptions(suppress=True, formatter={'float_kind':'{:16.6f}'.format}, linewidth=50)
np.set_printoptions(suppress=True, precision=6)

## What is a Neural Network?
### NN with 1 input

In [3]:
# The network:

weight = 0.1 
def neural_network(input, weight):
    prediction = input * weight
    return prediction

# How we use the network to predict something:

number_of_toes = [8.5, 9.5, 10, 9]
input = number_of_toes[0]
pred = neural_network(input,weight)
print(pred)

0.8500000000000001


### NN with multiple inputs

In [4]:
def w_sum(a,b):
    assert(len(a) == len(b))
    output = 0
    for i in range(len(a)):
        output += (a[i] * b[i])
    return output

weights = [0.1, 0.2, 0] 
    
def neural_network(input, weights):
    pred = w_sum(input,weights)
    return pred

# This dataset is the current
# status at the beginning of
# each game for the first 4 games
# in a season.

# toes = current number of toes
# wlrec = current games won (percent)
# nfans = fan count (in millions)

toes =  [8.5, 9.5, 9.9, 9.0]
wlrec = [0.65, 0.8, 0.8, 0.9]
nfans = [1.2, 1.3, 0.5, 1.0]

# Input corresponds to every entry
# for the first game of the season.

input = [toes[0],wlrec[0],nfans[0]]
pred = neural_network(input,weights)

print(pred)

0.9800000000000001


### Multiple Inputs - Numpy

In [5]:
a = [3, 4, 2]
b = [2, 1, 3]
result = np.dot(a, b)
result

16

In [6]:
import numpy as np
weights = np.array([0.1, 0.2, 0])
def neural_network(input, weights):
    pred = input.dot(weights)
    return pred
    
toes =  np.array([8.5, 9.5, 9.9, 9.0])
wlrec = np.array([0.65, 0.8, 0.8, 0.9])
nfans = np.array([1.2, 1.3, 0.5, 1.0])

# Input corresponds to every entry
# for the first game of the season.

input = np.array([toes[0],wlrec[0],nfans[0]])
pred = neural_network(input,weights)

print(pred)

0.9800000000000001


In [7]:
vector = [2, 3, 4]
lst = [0 for x in range(len(vector))]
lst

[0, 0, 0]

### Single Input - Multiple Output

In [8]:
def ele_mult(number, vector):
    output = [0 for x in range(len(vector))]
    for i in range(len(vector)):
        output[i] += number*vector[i]
        
    return output

def neural_network(predictor, weights):
    prediction = ele_mult(predictor, weights)
    
    return prediction

predictor = 0.65
weights = [0.3, 0.2, 0.8]

neural_network(predictor, weights) 

[0.195, 0.13, 0.52]

In [9]:
def neural_network(predictor, weights):
    prediction = np.dot(predictor, weights)
    
    return prediction

predictor = 0.65
weights = [0.3, 0.2, 0.8]

neural_network(predictor, weights) 

array([0.195, 0.13 , 0.52 ])

### Multiple Input and Output

In [10]:
def w_sum(a,b):
    assert(len(a) == len(b))
    output = 0
    for i in range(len(a)):
        output += (a[i] * b[i])
    return output

def vector_mat_mult(vector, matrix):
    output = [0 for x in range(len(matrix))]
    for i in range(len(matrix)):
        output[i] += w_sum(matrix[i], vector)
        
    return output

def neural_network(predictor, weights):
    prediction = vector_mat_mult(predictor, weights)
    
    return prediction

toes =  [8.5, 9.5, 9.9, 9.0]
wlrec = [0.65,0.8, 0.8, 0.9]
nfans = [1.2, 1.3, 0.5, 1.0]

predictor = [toes[0],wlrec[0],nfans[0]]
weights = np.array([[0.1, 0.1, -0.3], [0.1, 0.2, 0.0], [0.0, 1.3, 0.1]])

prediction = neural_network(predictor, weights)
prediction

[0.555, 0.9800000000000001, 0.9650000000000001]

In [11]:
predictor = [toes[0],wlrec[0],nfans[0]]
weights = np.array([[0.1, 0.1, -0.3], [0.1, 0.2, 0.0], [0.0, 1.3, 0.1], [0.1, 0.1, 0.1]]) # Different weight dimension 

prediction = neural_network(predictor, weights)
prediction

[0.555, 0.9800000000000001, 0.9650000000000001, 1.0350000000000001]

### Multiple Input and Output - with 1 Hidden layer

In [12]:
            #toes %win #fans
ih_wgt = [ [0.1, 0.2, -0.1], #hid[0]
           [-0.1,0.1, 0.9], #hid[1]
           [0.1, 0.4, 0.1] ] #hid[2]

           #hid[0] hid[1] hid[2]
hp_wgt = [ [0.3, 1.1, -0.3], #hurt?
           [0.1, 0.2, 0.0], #win?
           [0.0, 1.3, 0.1] ] #sad?

weights = [ih_wgt, hp_wgt]

def w_sum(a,b):
    assert(len(a) == len(b))
    output = 0
    for i in range(len(a)):
        output += (a[i] * b[i])
    return output

def vector_mat_mult(vector, matrix):
    output = [0 for x in range(len(matrix))]
    for i in range(len(matrix)):
        output[i] += w_sum(matrix[i], vector)
        
    return output

def neural_network(predictor, weights):
    hidden = vector_mat_mult(predictor, weights[0])
    output = vector_mat_mult(hidden, weights[1])
    return output

toes =  [8.5, 9.5, 9.9, 9.0]
wlrec = [0.65,0.8, 0.8, 0.9]
nfans = [1.2, 1.3, 0.5, 1.0]

# Input corresponds to every entry
# for the first game of the season.

predictor = [toes[0],wlrec[0],nfans[0]]
pred = neural_network(predictor,weights)

print(pred)

[0.21350000000000002, 0.14500000000000002, 0.5065]


In [13]:
            #toes %win #fans
ih_wgt = [ [0.1, 0.2, -0.1], #hid[0]
           [-0.1,0.1, 0.9], #hid[1]
           [0.1, 0.4, 0.1],
           [0.1, 0.0, 0.1]] #hid[2]

           #hid[0] hid[1] hid[2]
hp_wgt = [ [0.3, 1.1, -0.3, 0.1], #hurt?
           [0.1, 0.2, 0.0, 0.1]] #win?
           

weights = [ih_wgt, hp_wgt]

def w_sum(a,b):
    assert(len(a) == len(b))
    output = 0
    for i in range(len(a)):
        output += (a[i] * b[i])
    return output

def vector_mat_mult(vector, matrix):
    output = [0 for x in range(len(matrix))]
    for i in range(len(matrix)):
        output[i] += w_sum(matrix[i], vector)
        
    return output

def neural_network(predictor, weights):
    hidden = vector_mat_mult(predictor, weights[0])
    output = vector_mat_mult(hidden, weights[1])
    return output

toes =  [8.5, 9.5, 9.9, 9.0]
wlrec = [0.65,0.8, 0.8, 0.9]
nfans = [1.2, 1.3, 0.5, 1.0]

# Input corresponds to every entry
# for the first game of the season.

predictor = [toes[0],wlrec[0],nfans[0]]
pred = neural_network(predictor,weights)

print(pred)

[0.31050000000000005, 0.24200000000000005]


### Neural Network with 1 hidden layer - Numpy

In [14]:
import numpy as np

#toes %win #fans
ih_wgt = np.array([ 
            [0.1, 0.2, -0.1], #hid[0]
            [-0.1,0.1, 0.9], #hid[1]
            [0.1, 0.4, 0.1]]).T #hid[2]


# hid[0] hid[1] hid[2]
hp_wgt = np.array([  
            [0.3, 1.1, -0.3], #hurt?
            [0.1, 0.2, 0.0], #win?
            [0.0, 1.3, 0.1] ]).T #sad?

weights = [ih_wgt, hp_wgt]

def neural_network(input, weights):

    hid = input.dot(weights[0])
    pred = hid.dot(weights[1])
    return pred


toes =  np.array([8.5, 9.5, 9.9, 9.0])
wlrec = np.array([0.65,0.8, 0.8, 0.9])
nfans = np.array([1.2, 1.3, 0.5, 1.0])

input = np.array([toes[0],wlrec[0],nfans[0]])

pred = neural_network(input,weights)
print(pred)

[0.2135 0.145  0.5065]
